# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure the API Key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('..\Output_Data\Weather_Data_Collection.csv')
weather_df = weather_df.drop(columns={'Unnamed: 0'})
weather_df.head()



,City,Lattitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,butaritari,3.07,172.79,27.30,79,0,6.66,KI,1601273230
1,puerto ayora,36.59,-6.23,15.00,93,3,2.60,ES,1601273231
2,auki,-8.77,160.70,27.89,77,31,2.76,SB,1601273232
3,cherskiy,68.75,161.30,6.86,61,99,2.85,RU,1601273232
4,noumea,-22.28,166.46,21.00,88,90,6.20,NC,1601273233


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Obtaining the Coordinates to plot on the map
city_locations = weather_df[['Lattitude','Longitude']].astype(float)
city_locations.head()

,Lattitude,Longitude
0,3.07,172.79
1,36.59,-6.23
2,-8.77,160.70
3,68.75,161.30
4,-22.28,166.46


In [4]:
# Humidity values
humidity_data = weather_df['Humidity']
humidity_data.head()

0    79
1    93
2    77
3    61
4    88
Name: Humidity, dtype: int64

In [5]:
# Creating a Humidity Heatmap Layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(city_locations, weights=humidity_data, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(humidity_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# The most ideal vacations involve destinations with minimal cloud cover as well as having temperatures within the range of 20-30 degrees celcius. As such, there datframe will be narrowed down further in order to find hotels within these regions that have the ideal weather. 

ideal_weather_df = weather_df.loc[
    (weather_df['Cloudiness']==0) & 
    (weather_df['Wind Speed'] <= 10) & 
    (weather_df['Max Temp'] >= 20) &
    (weather_df['Max Temp'] <= 30)].dropna()

print(f' The Number of Ideal Cities within the ideal weather criteria are {len(ideal_weather_df)}')
ideal_weather_df


 The Number of Ideal Cities within the ideal weather criteria are 25


,City,Lattitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,butaritari,3.07,172.79,27.30,79,0,6.66,KI,1601273230
47,rikitea,-23.12,-134.97,23.63,81,0,4.79,PF,1601273261
80,ginda,30.69,78.49,29.00,62,0,2.23,IN,1601273284
93,katsuura,35.13,140.30,28.33,40,0,2.68,JP,1601273293
116,hun,29.13,15.95,29.25,18,0,0.66,LY,1601273307
142,kozluk,38.19,41.49,23.00,49,0,2.60,TR,1601273325
163,nouadhibou,20.93,-17.03,22.00,73,0,9.80,MR,1601273339
172,adrar,20.50,-10.07,28.77,22,0,2.83,MR,1601273345
205,kishtwar,33.32,75.77,24.32,19,0,1.78,IN,1601273370
238,tiznit,29.58,-9.50,20.97,50,0,0.39,MA,1601273393


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Creating a New Hotel DataFram
hotel_df = ideal_weather_df.copy().drop(columns={'Date', 'Wind Speed','Max Temp','Humidity','Cloudiness'})

hotel_df['Hotel Name'] = ""

hotel_df


,City,Lattitude,Longitude,Country,Hotel Name
0,butaritari,3.07,172.79,KI,
47,rikitea,-23.12,-134.97,PF,
80,ginda,30.69,78.49,IN,
93,katsuura,35.13,140.30,JP,
116,hun,29.13,15.95,LY,
142,kozluk,38.19,41.49,TR,
163,nouadhibou,20.93,-17.03,MR,
172,adrar,20.50,-10.07,MR,
205,kishtwar,33.32,75.77,IN,
238,tiznit,29.58,-9.50,MA,


In [8]:
# Setting up the parameters to determine the closest hotels within a 5 km radium (5000 m)

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Setting up the search parameters
params = {
    'type':'hotel',
    'keyword': 'hotel',
    'radius': 5000,
    'key': g_key
}

# We will be itterating through the rows in order to hotels within the defined radius of the coordinates of each city
print(f'Data retrival in progress')
for index, row in hotel_df.iterrows():
    latitude = row['Lattitude']
    longitude = row['Longitude']
    city = row['City']

    # Now that the latitude and longitude has been defined, these need to be added to the dictionary

    params['location'] = f'{latitude},{longitude}'

    
    response = requests.get(base_url, params=params).json()

    search_results = response['results']

    try:
        print(f'Finding nearest hotel for {city}\n--------------------------')
        hotel_df.loc[index, 'Hotel Name'] = search_results[0]['name']
    
    except (KeyError, IndexError):
        print(f'There are no Hotels for {city}, moving on to the next city....\n')


print('End of Data Retreival\n*********************')
print(f'Removing cities where hotels could not be retreived')
hotel_df = hotel_df[hotel_df['Hotel Name']!=""]




Data retrival in progress
Finding nearest hotel for butaritari
--------------------------
There are no Hotels for butaritari, moving on to the next city....

Finding nearest hotel for rikitea
--------------------------
Finding nearest hotel for ginda
--------------------------
Finding nearest hotel for katsuura
--------------------------
Finding nearest hotel for hun
--------------------------
Finding nearest hotel for kozluk
--------------------------
There are no Hotels for kozluk, moving on to the next city....

Finding nearest hotel for nouadhibou
--------------------------
Finding nearest hotel for adrar
--------------------------
There are no Hotels for adrar, moving on to the next city....

Finding nearest hotel for kishtwar
--------------------------
Finding nearest hotel for tiznit
--------------------------
There are no Hotels for tiznit, moving on to the next city....

Finding nearest hotel for castanos
--------------------------
Finding nearest hotel for asyut
-------------

In [9]:
# Results of the Hotel Search
hotel_df

,City,Lattitude,Longitude,Country,Hotel Name
47,rikitea,-23.12,-134.97,PF,Pension Maro'i
80,ginda,30.69,78.49,IN,Hotel Himalayan Inn
93,katsuura,35.13,140.30,JP,Splash Guest House
116,hun,29.13,15.95,LY,Alrwasi Hotel
163,nouadhibou,20.93,-17.03,MR,Free Zone Hotel Nouadhibou
205,kishtwar,33.32,75.77,IN,Hotel Phoenix International
280,castanos,26.78,-101.42,MX,Motel
316,asyut,27.18,31.18,EG,Assiut hotels Armed Forces
327,nadadores,27.05,-101.60,MX,Hotel Villa Jardín
379,awbari,26.59,12.78,LY,فندق أوباري


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lattitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
hotel_marker = gmaps.marker_layer(locations, info_box_content=hotel_info)

# We will add the new layer to the previous humidity heatmap
fig.add_layer(hotel_marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))